In [2]:
import numpy as np
from scipy import constants 
import matplotlib.pyplot as plt
import os

def fftconvolve(ax,ay,axis=-1):
    nex=ax.shape[axis]+ay.shape[axis]-1
    return np.fft.ifft(np.fft.fft(ax,n=nex,axis=axis)*np.fft.fft(ay,n=nex,axis=axis),axis=axis)

In [3]:
TS = 290        # noise temperature K
npulse = 16    # number of pulses in sequence
# nfft = 512      # zero padding in fft
Pt = 100        # peak power    W
G = 100         # Gain
F0 = 10.35e9       # carrier frequency Hz
wavelength = constants.c/F0     # radar wavelength  m
B=20e6
ts = 5e-9       # sample rate   s
PRF = 750e3     # PRF   Hz
PRT=1/PRF
Tx = 1/PRF      # pulse width
Va=constants.c*PRF/4/F0
Lx=Tx/ts
chirp = B/Tx
Ra=constants.c/PRF/2    # maximum unambiguous range
# print(Ra)
# rref = 3e3-500
rref = 0
# rswath = 1000
rswath = Ra
tref = 2*rref/constants.c
# tref = 0

de = wavelength/2
xx,yy = np.meshgrid(np.arange(-3.5,3.6,1),np.arange(-1.5,1.6,1))
element_loc0 = np.array([xx.flatten(),yy.flatten(),np.zeros(xx.size)]).transpose()*de

theta0 = 0 * np.pi/180  # azimuth y->x
phi0 = 0 * np.pi/180    #zenith
a0 = np.array([np.sin(phi0)*np.sin(theta0), np.sin(phi0)*np.cos(theta0), np.cos(phi0)])[:,np.newaxis,np.newaxis]

txp = np.arange(0,Tx,ts)    # discrete time of pulse
# create pulse window
tmpwd = np.hanning(txp.size//10)
wd = np.ones(txp.size)
wd[:tmpwd.size//2] = tmpwd[:tmpwd.size//2]
wd[-tmpwd.size//2:] = tmpwd[-tmpwd.size//2:]
tx = np.zeros(txp.shape,dtype=np.complex128)
tx[:tx.size//2] = Pt*wd[:tx.size//2]*np.exp((-np.pi*B*txp[:tx.size//2]+np.pi*chirp*txp[:tx.size//2]**2)*1j) # transmit waveform
txpl=txp[tx.size//2:]-txp[tx.size//2]
tx[tx.size//2:] = Pt*wd[tx.size//2:]*np.exp((np.pi*B*txpl-np.pi*chirp*txpl**2)*1j)
tf = np.arange(1e-5*ts,(2*rswath/constants.c)//ts*ts+1e-5*ts,ts)  # fast time
fast_time = tf[np.newaxis,:]
tm = np.arange(npulse)*PRT
tm = tm[:,np.newaxis]
Faxe=np.arange(-1/2/ts,1/2/ts,1/ts/tf.size)  # frequency after fft
Faxe = Faxe[np.newaxis,:]

In [ ]:
def fire_pulse(element_loct,target_p):
    total_rx = np.zeros((npulse,tf.size,int(element_loct.shape[0])),dtype=tx.dtype)
    for Rtar,Vtar,rcs, azt, elt in zip(*target_p):
        tau_0 = 2*Rtar/constants.c # first pulse target time delay
        FD = 2*Vtar/wavelength  # target doppler
        ARtar = constants.c*(tau_0-np.floor(tau_0/PRT)*PRT)/2 # apperent range of target
        Radar_c = np.sqrt(G**2*wavelength**2*rcs/(4*np.pi)**3)
        # *np.exp(2j*np.pi*np.random.random(1)) # radar constant without scale of range
        rr = constants.c*(tau_0)/2       # range
        Radar_c = Radar_c/rr**2     # return power
    
        rx=np.zeros(tf.shape,dtype=tx.dtype)       # received signal
        rx[:tx.size]=tx
    
        tau_m = 2*(Rtar-tm*Vtar)/constants.c-tref
        rx=np.tile(rx,(npulse,1))
        Radar_c=Radar_c
        frx=np.fft.fftshift(np.fft.fft(rx,axis=1),axes=1) 
            # frequency content of received signal
        dfrx=frx*np.exp(-2j*np.pi*Faxe*tau_m)   # time delay in frequency domain
    
        rx=np.fft.ifft(np.fft.fftshift(dfrx,axes=1))*Radar_c
        rx=rx*np.exp(2j*np.pi*FD*fast_time)*np.exp(-2j*np.pi*F0*tau_m)
        az = azt*np.pi/180
        ele = elt*np.pi/180
        targetRvector = np.array([np.sin(ele)*np.sin(az),np.sin(ele)*np.cos(az),np.cos(ele)])[np.newaxis,:]
        dl = np.sum(element_loct*targetRvector,axis=1)[np.newaxis,np.newaxis,:]
        total_rx=total_rx+rx[:,:,np.newaxis]*np.exp(-2j*np.pi*dl/wavelength)
    return total_rx

In [5]:
txpl=txp[tx.size//2:]-txp[tx.size//2]

In [4]:
tx.size

267